In [1]:
from __future__ import absolute_import
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import numpy as np
import random
import math
from preprocess import get_data_ocr
from sklearn import preprocessing
from ocr import *

2023-05-03 17:12:13.264817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [78]:
X_train, X_test, X_val, Y_train, Y_test, Y_val = get_data_ocr(.3, "./../data/")

In [79]:
char_encoder = preprocessing.LabelEncoder().fit(np.concatenate((Y_train.reshape(-1), Y_test.reshape(-1), Y_val.reshape(-1))))
ohe_size = np.max(char_encoder.transform(np.concatenate((Y_train.reshape(-1), Y_test.reshape(-1), Y_val.reshape(-1)))))
orig_shapes = [Y_train.shape[0], Y_test.shape[0], Y_val.shape[0]]
Y_train = char_encoder.transform(Y_train.reshape(-1)).reshape((orig_shapes[0], 4))
Y_test = char_encoder.transform(Y_test.reshape(-1)).reshape((orig_shapes[1], 4))
Y_val = char_encoder.transform(Y_val.reshape(-1)).reshape((orig_shapes[2], 4))

#Convert input to properly shaped and typed tensors
X_train  = tf.convert_to_tensor(np.asarray(X_train).astype(np.float32))
Y_train  = tf.convert_to_tensor(np.asarray(Y_train))
X_val  = tf.convert_to_tensor(np.asarray(X_val).astype(np.float32))
Y_val  = tf.convert_to_tensor(np.asarray(Y_val))

#Don't reshape test, because we need to preserve CAPTCHAs but convert to tensor
X_test  = tf.convert_to_tensor(np.asarray(X_test).astype(np.float32))
Y_test  = tf.convert_to_tensor(np.asarray(Y_test))

#Expand Dims
X_train = tf.expand_dims(X_train, axis=-1)
X_val   = tf.expand_dims(X_val  , axis=-1)
X_test  = tf.expand_dims(X_test , axis=-1)

In [80]:
input_shape=(X_train.shape[-3], X_train.shape[-2], X_train.shape[-1])
print(input_shape)


def ctc(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    #input_length = tf.math.reduce_sum(y_pred, axis=-1, keepdims=False)
    #label_length = tf.math.count_nonzero(y_true, axis=-1, keepdims=False, dtype=tf.int32)

    return tf.keras.backend.ctc_batch_cost(y_true, y_pred, label_length=label_length, input_length=input_length)
# Compile and fit model
model = create_model(input_shape, (ohe_size+1))
model.compile(loss=ctc,
                optimizer=tf.keras.optimizers.Adam(.0001))

model.summary()

(24, 72, 1)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 24, 72, 32)        320       
                                                                 
 batch_normalization_3 (Batc  (None, 24, 72, 32)       128       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 24, 72, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 12, 36, 64)        18496     
                                                                 
 batch_normalization_4 (Batc  (None, 12, 3

In [5]:
model.fit(
    X_train,
    Y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, Y_val)
)

# Save model for future testing



Epoch 1/10
28/28 [==============================] - 30s 841ms/step - loss: 15.1904 - val_loss: 15.0795
Epoch 2/10
28/28 [==============================] - 24s 845ms/step - loss: 14.6345 - val_loss: 15.3893
Epoch 3/10
28/28 [==============================] - 28s 993ms/step - loss: 13.5953 - val_loss: 17.7645
Epoch 4/10
28/28 [==============================] - 27s 987ms/step - loss: 11.8382 - val_loss: 20.4704
Epoch 5/10
28/28 [==============================] - 27s 964ms/step - loss: 10.0397 - val_loss: 20.0012
Epoch 6/10
28/28 [==============================] - 27s 982ms/step - loss: 8.5861 - val_loss: 18.3420
Epoch 7/10
28/28 [==============================] - 28s 1s/step - loss: 7.4738 - val_loss: 15.6496
Epoch 8/10
28/28 [==============================] - 27s 971ms/step - loss: 6.5056 - val_loss: 13.5687
Epoch 9/10
28/28 [==============================] - 27s 968ms/step - loss: 5.7871 - val_loss: 11.3339
Epoch 10/10
28/28 [==============================] - 26s 946ms/step - loss: 5.12

In [6]:
model.save('./../models/crnn', save_format="h5")

In [7]:
loaded_model = tf.keras.models.load_model("./../models/crnn", custom_objects={'ctc': ctc})
print(loaded_model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 72, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 24, 72, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 24, 72, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 36, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 36, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 12, 36, 64)       2

In [8]:

# Run random CAPTCHA test
output = model.predict(X_test[1:2], verbose=0)
print("Secret CAPTCHA:")
print(char_encoder.inverse_transform(Y_test[1]))
print("Model guess:")
output = np.transpose(output, axes=[1, 0, 2]).squeeze()
print(output)
print(np.argmax(output, axis=1))
# _, max_index = np.max(output, axis=2)
#raw_prediction = list(max_index[:, i].detach().cpu().numpy())
prediction = tf.unique_with_counts(np.argmax(output, axis=1))
print(prediction)
# print(char_encoder.inverse_transform(output))

# # Print individual and captcha-based accuracy
# indiv_acc, group_acc = run_tests(model, X_test, Y_test)
# print(f"Individual accuracy: {round(100*indiv_acc, 2)}% \nCaptcha accuracy: {round(100*group_acc, 2)}%")

Secret CAPTCHA:
['B' 'S' 'H' 'F']
Model guess:
[[1.88498339e-03 1.76497802e-01 4.24088724e-03 6.96901605e-02
  4.57353890e-03 1.80103758e-04 3.99093449e-01 6.96475850e-03
  5.48067596e-03 1.11697294e-01 2.75980739e-04 4.44790523e-04
  9.24414769e-02 2.13075359e-03 7.67696090e-03 1.90014821e-02
  5.43603637e-05 9.33453150e-04 1.83832797e-03 3.47220019e-04
  2.45293276e-03 1.99892209e-03 4.02391015e-04 1.04253236e-02
  1.45716369e-02 1.48199976e-03 3.71329440e-03 1.39816885e-03
  5.41738467e-04 3.33411992e-02 1.31473318e-03 5.54270577e-03
  1.73665229e-02]
 [7.47421349e-04 7.49142468e-02 1.36918505e-03 3.73440385e-02
  1.99922686e-03 5.70560151e-05 2.35844001e-01 2.59431731e-03
  4.20288043e-03 1.16405070e-01 2.08055571e-04 8.64907284e-04
  3.09866995e-01 1.80087413e-03 1.15534933e-02 6.28343597e-02
  2.42319529e-05 1.26413058e-03 1.61501521e-03 3.61018727e-04
  3.34855542e-03 1.24845735e-03 6.76307769e-04 6.86017936e-03
  9.66955628e-03 8.54055514e-04 1.08130788e-02 1.14145223e-03
  1.0

In [85]:
#OCR Accuracy

def get_accuracy(model, X_test, Y_test):
    print(X_test.shape)
    print(Y_test.shape)
    probs = model.predict(X_test, verbose=0)
    output = np.argmax(probs, axis=2)
    print(output[0:10])
    prediction = np.asarray([np.asarray(tf.unique_with_counts(x)[0]) for x in output])
    print(prediction[0].shape)
    Y_test = np.asarray(Y_test)
    count = 0
    total = Y_test.shape[0]
    for x, y in zip(prediction, Y_test):
        if len(x) == len(y) and np.all(np.equal(x, y)): count += 1
    accuracy = count / total

    return accuracy

In [86]:
model = tf.keras.models.load_model("./../models/ocr_basic", custom_objects={'ctc': ctc})
get_accuracy(model, X_test, Y_test)

(1493, 24, 72, 1)
(1493, 4)
[[19 19 19 19 28 28]
 [19 19 19 19 19 28]
 [19 19 19 19 19  9]
 [19 19 19 19  9  9]
 [19 19 19 28 28 28]
 [19 19 19 19 28  9]
 [19 19 19 19 19 28]
 [19 19 19 19 19 28]
 [19 19 19 19 28 28]
 [19 19 19 19 19 28]]
(2,)


/var/folders/0x/2kts5pgd6cs9gk2hysn_90br0000gn/T/ipykernel_5482/1384438526.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prediction = np.asarray([np.asarray(tf.unique_with_counts(x)[0]) for x in output])


0.0